In [1]:
import cv2
from PyQt5.QtGui import QPixmap, QImage
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QLineEdit, QPushButton, QRadioButton, QGridLayout, QTimeEdit,\
    QVBoxLayout, QButtonGroup, QDateTimeEdit, QGraphicsView, QGraphicsScene

from PyQt5.QtCore import QDateTime,QTimer,Qt

from yolo_8_security import ObjectDetection


class HomeApp(QWidget, ):
    def __init__(self):
        super().__init__()
        self.initUI()

        # 수정된 setup video capture
        
        self.detector = ObjectDetection(capture_index=0, frame_width=640, frame_height=480)
        
        # update_frame을 30ms마다 호출
        self.timer_for_update_frame = QTimer(self)
        self.timer_for_update_frame.timeout.connect(self.update_frame)
        self.timer_for_update_frame.start(30)

        # set_current_time 를 1s마다 호출
        self.timer_for_set_current_time = QTimer(self)
        self.timer_for_set_current_time.timeout.connect(self.set_current_time)
        self.timer_for_set_current_time.start(1000)
        
        
        # 들어간사람&나간사람멤버변수
        self.in_counts= 0
        self.out_counts = 0


    def initUI(self):
        self.grid = QGridLayout()
        self.setLayout(self.grid)

        # 첫 번째 행: 현재 전력량, 입력 박스, 입력 버튼
        self.grid.addWidget(QLabel('현재 전력량'), 1, 1)
        self.grid.addWidget(QLineEdit(), 1, 2)
        self.grid.addWidget(QPushButton('input'), 1, 3)
    
        # 에어컨 행 라디오 버튼 그룹
        self.radio_group1 = QButtonGroup(self)
  
        radio_on1 = QRadioButton('ON')
        radio_off1 = QRadioButton('OFF')
        radio_on1.setEnabled(True)  # 활성화
        radio_off1.setEnabled(True)  # 활성화
        self.radio_group1.addButton(radio_on1)
        self.radio_group1.addButton(radio_off1)
        self.grid.addWidget(QLabel('에어컨'), 2, 1)
        self.grid.addWidget(radio_on1, 2, 2)
        self.grid.addWidget(radio_off1, 2, 3)
    
        # 보일러 행 라디오 버튼 그룹
        self.radio_group2 = QButtonGroup(self)
        radio_on2 = QRadioButton('ON')
        radio_off2 = QRadioButton('OFF')
        radio_on2.setEnabled(True)  # 활성화
        radio_off2.setEnabled(True)  # 활성화
        self.radio_group2.addButton(radio_on2)
        self.radio_group2.addButton(radio_off2)
        self.grid.addWidget(QLabel('보일러'), 3, 1)
        self.grid.addWidget(radio_on2, 3, 2)
        self.grid.addWidget(radio_off2, 3, 3)
    
        # 전등 행 라디오 버튼 그룹
        self.radio_group3 = QButtonGroup(self)  
        radio_on3 = QRadioButton('ON')   
        radio_off3 = QRadioButton('OFF')
        radio_on3.setEnabled(True)  # 활성화
        radio_off3.setEnabled(True)  # 활성화
        self.radio_group3.addButton(radio_on3)
        self.radio_group3.addButton(radio_off3)
        self.grid.addWidget(QLabel('전등'), 4, 1)
        self.grid.addWidget(radio_on3, 4, 2)
        self.grid.addWidget(radio_off3, 4, 3)
        
        # QDateTimeEdit 섹션
        lbl = QLabel('날짜 및 시간 선택:')
        self.datetimeedit = QDateTimeEdit(self)  # 날짜 및 시간 선택 위젯

        self.datetimeedit.setDisplayFormat('yyyy-MM-dd HH:mm:ss') 
        
        # QVBoxLayout에 QDateTimeEdit 추가
        self.vbox_widget = QWidget()
        vbox = QVBoxLayout(self.vbox_widget)
        vbox.addWidget(lbl)
        vbox.addWidget(self.datetimeedit)
        vbox.addStretch()
        # 조회 버튼 추가
        self.queryButton = QPushButton('조회', self)
        self.queryButton.clicked.connect(self.on_query_clicked)  # 클릭 이벤트 연결
        vbox.addWidget(self.queryButton)
        # v박스를 전체레이어에 추가
        self.grid.addWidget(self.vbox_widget,5, 1)
        
        # 비디오 스트리밍을 위한 QLabel 추가
        self.video_label = QLabel(self)
        self.video_label.resize(640, 480)
        self.grid.addWidget(self.video_label, 6, 1, 1, 4) # 비디오 라벨을 그리드 레이아웃에 추가

        # 윈도우 설정
        self.setGeometry(50, 50, 700, 800)
        self.setWindowTitle('Home')
        self.show()
        
        ### 로직추가
        # incount, outcount로 incount-outcount 가 <=0 이면 전등 off로직 구현 

    def update_frame(self):
        ret, frame = self.detector.cap.read()
        if not ret:
            return  # 프레임을 읽지 못하면 함수 종료
        # 생성된 카운터 클래스로 다시 모델 카운트 하기
        self.in_counts, self.out_counts, processed_frame = self.detector.model_count()
        
        if processed_frame is not None:
            height, width, channel = processed_frame.shape
            qimage = QImage(processed_frame.data, width, height, QImage.Format_RGB888).rgbSwapped()
            self.video_label.setPixmap(QPixmap.fromImage(qimage))

        # 새로운 QWidget 인스턴스를 그리드 레이아웃에 추가
        self.grid.addWidget(self.vbox_widget, 5, 1)

    def on_query_clicked(self):
        # 선택된 날짜 및 시간을 문자열 형식으로 변환
        selected_datetime = self.datetimeedit.dateTime().toString("yyyy-MM-dd HH:mm")

        # MariaDB에서 상태 가져오기
        pc = PowerClass()
        rs = pc.select(f"""
                select * from homedevice
                where time = '{selected_datetime}';
        """)
        # 에어컨 상태 설정
        if rs['airconditioner'][0] == True:
            self.radio_group1.buttons()[0].setChecked(True)  # ON
        else:
            self.radio_group1.buttons()[1].setChecked(True)  # OFF

        # 보일러 상태 설정7
        if rs['heater'][0] ==  True:
            self.radio_group2.buttons()[0].setChecked(True)  # ON
        else:
            self.radio_group2.buttons()[1].setChecked(True)  # OFF

        # 전등 상태 설정
        if rs['light'][0] == True:
            self.radio_group3.buttons()[0].setChecked(True)  # ON
        else:
            self.radio_group3.buttons()[1].setChecked(True)  # OFF
    
    def set_current_time(self):
        Qtime = QDateTime.currentDateTime()
        Qtime = Qtime.addSecs(60*60*9) #60초*60분*9시간
        self.datetimeedit.setDateTime(Qtime)
        
    def release_camera(self):  # 카메라 자원을 반납하고 종료하는 함수
        self.detector.cap.release()


ModuleNotFoundError: No module named 'yolo_8_security'

In [2]:
import pandas as pd
import pymysql

class PowerClass:
    def __init__(self):
        pass
    # 시작
    def start(self):
        try:
            self.conn = pymysql.connect(host = "35.87.206.219",
                                        user = "Jada",
                                        password = "think2na",
                                        db = "Jada",
                                        charset = "utf8",
                                        autocommit = True,
                                        cursorclass = pymysql.cursors.DictCursor)
            print("DB 접속 성공", self.conn)
        except:
            print("DBserver check...")
        self.cur = self.conn.cursor()
        
    
    # 조회
    def select(self, sql):
        self.start()
        rs_cnt = self.cur.execute(sql)
        
        if rs_cnt >1:
            print(f'{rs_cnt}건 조회')
            read = self.cur.fetchall()
            df = pd.DataFrame(read)
            self.exit()
            return df
        elif rs_cnt == 1:
            print(f'{rs_cnt}건 조회')
            read = self.cur.fetchall() #list(self.cur.fetchone())+  list(self.cur.fetchone().values())  
            print(read)
            df = pd.DataFrame(read)
            self.exit()
            return df
        else: print('조회될건이 없습니다.')
        self.exit()
        
    # 입력
    def insert(self,sql):
        self.start()
        rs_cnt = self.cur.execute(sql)
        if rs_cnt>=1:
            print(f'{rs_cnt}건 입력됨')
        else:
            print('입력된 것이 없거나 오류')
        self.exit()
        
    # 수정
    def update(self, sql):
        self.start()
        rs_cnt = self.cur.execute(sql)
        if rs_cnt>=1:
            print(f'{rs_cnt}건 수정됨')
        else:
            print('수정된 것이 없거나 오류')
        self.exit()
        
    # 삭제
    def delete(self,sql):
        self.start()
        rs_cnt = self.cur.execute(sql)
        if rs_cnt>=1:
            print(f'{rs_cnt}건 삭제됨')
        else:
            print('삭제된 것이 없거나 오류')
        self.exit()
        
    # 종료
    def exit(self):
        #접속종료
        try:
            print('접속해제')
            self.cur.close()
            self.conn.close()
        except:
            print("이미 꺼짐")
            

In [3]:
def main():
    app = QApplication(sys.argv)
    ex = HomeApp()
    app.aboutToQuit.connect(ex.release_camera)
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

Region Counter Initiated.


SystemExit: 0

/Users/taeyoon/anaconda3/envs/gj_env_jada/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
